In [1]:
import json
import pickle
from helper_function import preprocess, Indexer
from elasticsearch import Elasticsearch

In [2]:
!which python

/c/Users/junec/AppData/Local/Programs/Python/Python38/python


In [3]:
!which pip

/c/Users/junec/AppData/Local/Programs/Python/Python38-32/Scripts/pip


In [ ]:
#conda install elasticsearch

**read from short_abstracts_en.ttl and covert to map with key as the name of entity, value as abstract**

In [4]:
INDEX_NAME = "dbpedia_entity_centric"
INDEX_SETTINGS = {
    "mappings": {
        "properties": {
            "entity": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },
            "abstract": {
                "type": "text",
                "term_vector": "yes",
                "analyzer": "english",
            },

        }
    }
}
es = Elasticsearch()
es.info()

c:\users\junec\appdata\local\programs\python\python38-32\lib\site-packages\elasticsearch\connection\base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

**indexing finished, do not run the following cell**

In [5]:
# #test file
# #filepath="../data/xaa_small_abs"

# filepath="../data/short_abstracts_en.ttl"
# entity_map_abstract={}
# #collections=[]
# i=0
# j=0
# with open(filepath, 'r', encoding='utf-8') as file: 
#     for line in file:
#         j+=1
#         if j%100000==0:
#             print("-----------j",j)
#         if (line is None) or (line[0] == '#'):
#             continue
#         #line = line.lower().strip()[:-5].replace('/>', '>').split(' ')
#         line=line.split(' ')
#         if len(line) < 3:
#             continue
#         entity = line[0][1:-1].split('/')[-1]
#         entity=preprocess(entity)
#         line[-2]=line[-2][:-4]
#         abstract = preprocess(' '.join(line[2:-1]).replace('\\', ''))
#         #entity_map_abstract[entity]=abstract
#         #collections.append({"id":str(i),"entity":entity,"abstract":abstract})
#         doc={"entity":entity,"abstract":abstract}
#         es.index(index=INDEX_NAME, doc_type="_doc", id=str(i), body=doc)
#         if i%100000==0:
#             print("-----------i",i)
#         i+=1
# print("----i",i)
# print("----j",j)

C:\Users\junec\AppData\Local\Temp/ipykernel_16056/3275183715.py:27: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME, doc_type="_doc", id=str(i), body=doc)


-----------i 0
-----------j 100000
-----------i 100000
-----------j 200000
-----------i 200000
-----------j 300000
-----------i 300000
-----------j 400000
-----------i 400000
-----------j 500000
-----------i 500000
-----------j 600000
-----------i 600000
-----------j 700000
-----------i 700000
-----------j 800000
-----------i 800000
-----------j 900000
-----------i 900000
-----------j 1000000
-----------i 1000000
-----------j 1100000
-----------i 1100000
-----------j 1200000
-----------i 1200000
-----------j 1300000
-----------i 1300000
-----------j 1400000
-----------i 1400000
-----------j 1500000
-----------i 1500000
-----------j 1600000
-----------i 1600000
-----------j 1700000
-----------i 1700000
-----------j 1800000
-----------i 1800000
-----------j 1900000
-----------i 1900000
-----------j 2000000
-----------i 2000000
-----------j 2100000
-----------i 2100000
-----------j 2200000
-----------i 2200000
-----------j 2300000
-----------i 2300000
-----------j 2400000
-----------i 240

In [ ]:
# es = Elasticsearch()
# es.info()


In [ ]:
# a_file = open("DBpedia_map_entity_abstract_1.pkl", "wb")
# pickle.dump(entity_map_abstract, a_file)
# a_file.close()

In [6]:
# INDEX_NAME = "dbpedia_entity_centric"
count = es.cat.count(INDEX_NAME, params={"format": "json"})
print(int(count[0]["count"]))
es.get(index=INDEX_NAME,id="15")

C:\Users\junec\AppData\Local\Temp/ipykernel_16056/2731474974.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  count = es.cat.count(INDEX_NAME, params={"format": "json"})


4935279


{'_index': 'dbpedia_entity_centric',
 '_type': '_doc',
 '_id': '15',
 '_version': 3,
 '_seq_no': 12013,
 '_primary_term': 1,
 'found': True,
 '_source': {'entity': 'Appellate procedure in the United States',
  'abstract': ' United States appellate procedure involves the rules and regulations for filing appeals in state courts and federal courts  The nature of an appeal can vary greatly depending on the type of case and the rules of the court in the jurisdiction where the case was prosecuted  There are many types of standard of review for appeals  such as de novo and abuse of discretion  However  most appeals begin when a party files a petition for review to a higher court for the purpose of overturning the lower court s decision '}}

In [7]:
term="appellate" 
field="abstract"
hits = (
        es.search(
            index=INDEX_NAME,
            query={"match": {field: term}},
            _source=False,
            size=1,
        )
        .get("hits", {})
        .get("hits", {})
    )
hits

[{'_index': 'dbpedia_entity_centric',
  '_type': '_doc',
  '_id': '714513',
  '_score': 13.31942}]

/c/Users/junec/AppData/Local/Programs/Python/Python38/python
0


C:\Users\junec\AppData\Local\Temp/ipykernel_16056/1254572629.py:2: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists("dbpdiea_type_centric")
C:\Users\junec\AppData\Local\Temp/ipykernel_16056/1254572629.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  count = es.cat.count("dbpdiea_type_centric", params={"format": "json"})


NotFoundError: NotFoundError(404, '{"_index":"dbpdiea_type_centric","_type":"_doc","_id":"15","found":false}')

In [ ]:
# with open("../data/DBpedia_map_entity_abstract.json", 'w',encoding='utf-8') as f:
#   json.dump(entity_map_abstract, f, ensure_ascii=False)